# CSPB 3202 Final Project #

Tyler Kinkade, Tyler.Kinkade@colorado.edu    

_All rights reserved_

GitHub: [https://github.com/jaki9292/rl-project](https://github.com/jaki9292/rl-project)

## Overview ##

This write-up reports on a small project to compare the effectiveness of artificial agent algorithms ranging in sophistication from naive to deep reinforcement learning (Russell & Norvig, 2022; Sutton & Barto, 2018) to successfully land a "lunar lander" agent in a Gymnasium (2022) model environment.

The report is divided into the following sections: approach, results, discussion, and suggestions.

## Approach ##

This section is divided into the following subsections: environment, models, methods.

### Environment ###

The lunar lander environment (pictured below) simulates rocket trajectory physics with the primary aim of landing a lunar lander on a central landing pad (marked by two flags) by means of turning its three rockets thrusters (left, right, and main) on or off. The agent operating the lander is rewarded for landing slowly, in an upright position, on both legs, on or near the landing pad and penalized otherwise. Fuel is unlimited, but a penalty is given for each time an engine fires. Scores of 200 points or more are considered a solution. 

The state space is described by an 8-vector comprised of the lander coordinates $(x,y)$, its linear velocities $(v_x,v_y)$ in the $x$ and $y$ directions, its angle $(\theta)$, its angular velocity $(\omega)$, and two Boolean variables $(l,r)$ that encode whether the left and right legs are in contact with the ground. The environment has both discrete and continuous versions, but only the discrete version is used here, in order to include simple models in the comparison. The environment gravity, wind power, and turbulence can also be specified. The starting state is the top center of the space with a random force applied to the lander. The termination state occurs when the lander stops moving or moves outside the frame. The action space is comprised of 4 discrete actions: do nothing, fire left thruster, fire main down thruster, or fire right thruster.

In [1]:
# Display Lunar Lander GIF
from IPython.display import Image
Image(url= "https://gymnasium.farama.org/_images/lunar_lander.gif")

In [ ]:
# Install dependencies
# %%bash
# !pip3 install gymnasium
# !pip3 install gymnasium[box2d]
# !pip3 install moviepy
# !pip3 install numpy
# !pip3 install pandas
# !pip3 install matplotlib
# !pip3 install torch

In [2]:
# Display environment parameters
# References: 
# https://gymnasium.farama.org/api/env/
# https://gymnasium.farama.org/environments/box2d/lunar_lander/

import gymnasium as gym

# Initialize environment
env = gym.make("LunarLander-v2", 
               continuous = False,      # Discrete version
               gravity = -10.0, 
               enable_wind = False, 
               wind_power = 0.0, 
               turbulence_power = 0.0, 
               render_mode="rgb_array") # Render for machine

# Reset environment with random number generator seed for reproducibility
state, info = env.reset(seed = 21)

print("Environment metadata:\n", env.metadata)
print("\nState space:\n", env.observation_space)
print("\nAction space:\n", env.action_space)
print("\nStarting state:\n", state)

Environment metadata:
 {'render_modes': ['human', 'rgb_array'], 'render_fps': 50}

State space:
 Box([-90.        -90.         -5.         -5.         -3.1415927  -5.
  -0.         -0.       ], [90.        90.         5.         5.         3.1415927  5.
  1.         1.       ], (8,), float32)

Action space:
 Discrete(4)

Starting state:
 [ 0.00786695  1.4032904   0.796822   -0.33911794 -0.00910905 -0.18049194
  0.          0.        ]


The table below summarizes the descriptive statistics for 100 randomly sampled initial states for this environment. We can see that the initial linear and angular velocities vary more than the position and angular orientation.

In [5]:
import pandas as pd

# Determine average starting state from 100 random samples
starting_states = []
for _ in range(100):
    starting_state, info = env.reset()
    starting_states.append(starting_state)

features = ['x','y','vx','vy','angle','ang_vel','left_leg','right_leg']
starting_states_df = pd.DataFrame(starting_states, columns = features)
starting_states_df.describe()

,x,y,vx,vy,angle,ang_vel,left_leg,right_leg
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.0
mean,-0.000202,1.410030,-0.020460,-0.039560,0.000241,0.004635,0.0,0.0
std,0.004445,0.007046,0.450270,0.313165,0.005151,0.101993,0.0,0.0
min,-0.007806,1.398572,-0.790636,-0.548822,-0.009096,-0.180236,0.0,0.0
25%,-0.003390,1.404051,-0.343335,-0.305285,-0.003875,-0.076865,0.0,0.0
50%,-0.000128,1.409595,-0.012973,-0.058933,0.000155,0.002939,0.0,0.0
75%,0.003350,1.416666,0.339339,0.255355,0.003934,0.077771,0.0,0.0
max,0.007856,1.421770,0.795693,0.482209,0.009052,0.179091,0.0,0.0


### Models ###
Four models are compared in this project: an agentless lunar lander, a random agent, a simple reflex agent, and a deep Q-network agent. 

#### Agentless Lunar Lander ####
An agentless lunar lander takes no action and is at the complete mercy of its environment. Because the starting state is directly above the landing pad, this might not be as terrible a strategy as it initially seems. The lander will simply fall to the ground in a trajectory determined by the physics of the environmental forces and the initial force placed upon it.

#### Random Agent ####

A random agent selects any of the possible actions uniformly randomly at each step without regard to the environment. The animation above is an example of such an agent and appears to be a poor strategy. Is this a worse strategy than an agentless lander though?

#### Simple Reflex Agent ####

A simple reflex agent can only react in a fixed way to sensory input but does not retain any memory of the past and, as a result, cannot learn (Russell & Norvig, 2022). As explained in the environment section, the initial lunar lander position is generally centered above the landing pad, but there is wide variation in initial linear and angular velocity. Thus, the primary task is counteracting any horizontal velocity while at the same time, avoiding turning the ship on its side. Therefore, there is a balance to be achieved, and the programmer must select appropriate threshold constants for the algorithm. Depending on the algorithm and constants, this could potentially be a better strategy than doing nothing or acting randomly.

#### Deep Q-Network Agent ####

A deep Q-network (DQN; Mnih et al., 2015, 2016) agent is a deep-learning neural network approximation of a Q-learning agent (Russell & Norvig, 2022; Sutton & Barto, 2018). A Q-learning agent alternates between learning through exploration and exploiting what it has learned by choosing the action with maximal value for each state. Although this is obviously better than the relatively ignorant reflex agent, Q-learning agents can quickly become computationally expensive and slow to learn in large and complex environments (Russell & Norvig, 2022; Sutton & Barto, 2018). 

Approximate Q-learning agents address this problem by approximating the Q-value of actions in particular states with a weighted feature function based on states and actions (Russell & Norvig, 2022; Sutton & Barto, 2018). However, in the environment in this project, we only have access to the current features and do not know how any particular action will influence those features. 

Fortunately, the universal approximation theorem tells us that we can approximate any function with a neural network of at least two layers  (Russell & Norvig, 2022). Thus, we can approximate a Q-value function with a DQN by recording a log of previous attempts (i.e., replay buffer) to train the network parameters (Paske & Towers, 2023). In other words, the agent remembers and learns from its experiences.

For this project, the DQN consists of three fully-connected neural network layers. A non-linear activation function is placed between the layers to prevent the model from collapsing into a one-layer linear model. Here, a rectified linear unit (ReLU) function is selected for this because of its computational simplicity and its consistent, demonstrated effectiveness in previous studies (Goodfellow et al., 2016; Karpathy, 2016). The input to the network are the eight state features and the output is a vector of approximate Q-values for each of the four possible actions given the input state. 

It is worth noting that image-based convolutional neural networks are frequently used for DQNs (e.g., Mnih et al., 2015). However, I am interested here in solving the lunar landing problem from the perspective of the sensory information available to an agent within the lunar lander, rather than an outside observer.

Another design choice is how often the agent should explore versus how often it should exploit what it has learned thus far. For this project, I will use an epsilon greedy policy which prioritizes learning at the beginning but gradually shifts to prioritizing achieving the highest rewards by selecting the policy it has learned thus far.

To train the model, I will use the [Huber loss](https://en.wikipedia.org/wiki/Huber_loss) function (Huber, 1964), which is more robust to noisy data than mean-squared error for training the model (Paske & Towers, 2023).For optimization, I will use the [Adam optimization algorithm](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/). Unlike the
[stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) (SGD), Adam optimization maintains and adapts separate learning rates for each parameter (Brownlee, 2017).

It is expected that the DQN model will perform better than the first three; however, it is still possible that some difficulties might arise in creating a suitable neural network.

### Methods ###

Each model will be run for 1000 episodes and their scores over each of the episodes will be plotted and compared. Visualizing each model's performance over successive episodes permits comparison of how each agent progressed in learning how to solve the model. As Machado and his colleagues (2018) point out, this is preferable to single metrics, such as maximum or average scores which can conceal important distinctions. For example, we can discover whether an agent achieved a maximum only once, but performed poorly most of the time. Similarly, an average score can conceal unstable, fluctuating performance. Ideally, we would like to see an agent learning quickly and then stably maintaining high scores above the solution threshold of 200 points.

Applying this methodology, we will be able to explore the advantages and disadvantages of the various models and also demonstrate how a solution to the problem can be explored through the modfication of features and comparison of models.

In the following code blocks, I construct the Python classes required to encode each of the agents.

#### Agentless Lunar Lander ####
An agentless lunar lander which never fires any of its thrusters:

In [ ]:
# Define general lunar lander class
# Adapted from: 
# CSPB 3202 Gym tutorial (https://applied.cs.colorado.edu/mod/folder/view.php?id=43136)
# https://gymnasium.farama.org/
# https://gymnasium.farama.org/api/utils/#save-rendering-videos
# https://gymnasium.farama.org/environments/box2d/lunar_lander/
# https://github.com/openai/gym/blob/master/gym/envs/box2d/lunar_lander.py
# https://gymnasium.farama.org/tutorials/training_agents/FrozenLake_tuto/

from gymnasium.utils.save_video import save_video
import numpy as np
from IPython.utils.capture import capture_output

# Action space
NOOP, LEFT, MAIN, RIGHT = 0, 1, 2, 3

class LunarLander:
    '''
    General lunar lander class 
    subject entirely to its environment
    '''   
    def __init__(self, name = "agentless"):
        '''
        Initialize agent
        '''
        self.name = name
        self.env = gym.make("LunarLander-v2", 
               continuous = False,           # Discrete version
               gravity = -10.0, 
               enable_wind = False, 
               wind_power = 0.0, 
               turbulence_power = 0.0, 
               render_mode="rgb_array_list") # Render for machine
        
        # Reset environment with random number generator seed for reproducibility
        self.state, self.info = self.env.reset(seed = 21)
        self.features_size = len(self.state)
        self.actions_size = env.action_space.n
        self.scores = []
        self.steps = []
        self.policy = []
        
    def select_action(self, state):
        '''
        Select action given state
        '''
        # Do nothing
        return NOOP

    def episode(self):
        '''
        Complete episode in given environment
        Return score, steps count, and actions log
        '''
        # Initialize rewards total
        score = 0.0

        # Initialize steps count
        step_count = 0

        # Initialize actions list
        actions = []

        # Start new episode  
        state, info = self.env.reset()

        # Until end of episode
        while True:
            # Get action based on current state
            action = self.select_action(state)

            # Obtain next state, reward, terminated status, truncated status, 
            # and environment info for given action
            next_state, reward, terminated, truncated, info = self.env.step(action)
            
            # Accumulate reward total
            score += reward

            # Increment steps count
            step_count += 1

            # Record action
            actions.append(action)

            # If end of episode
            if terminated or truncated:
                break  # out of loop
            
            # Advance to next state 
            state = next_state

        return score, step_count, actions
    
    def iterate(self, n = 100, verbose = False, video = False):
        '''
        Attempt task for n episodes
        '''
        # Initialize logs
        scores = []
        steps = []
        policy = []

        video_step_start = 0
        
        # Repeat n times
        for episode in range(n):
            # Complete one episode
            score, step_count, actions = self.episode()
            
            # Log scores, steps, policy
            scores.append(score)
            steps.append(step_count)
            policy.append(actions)

            # Update object attributes
            self.scores = scores
            self.steps = steps
            self.policy = policy

            if verbose:
                # Report result
                print(f"Episode {episode} score: {score}")

            if video:
                # Suppress MoviePy stdout
                # https://stackoverflow.com/a/35624406/14371011
                with capture_output() as captured:
                    # Save mp4 of every (k^3)th episode (0, 1, 8, 27, ...)
                    save_video(self.env.render(),
                               video_folder = "videos",
                               name_prefix = self.name,
                               fps = env.metadata["render_fps"],
                               step_starting_index = video_step_start,
                               episode_index = episode)

            # Advance starting step index for videos
            video_step_start = step_count + 1           
        
        return scores
    
    def score_stats(self, verbose = False):
        '''
        Return mean, max, and min of scores if they exist 
        '''
        # If scores exist
        if (len(self.scores) > 0):
            # Calculate average, max, and min
            average, max, min = np.average(self.scores), np.max(self.scores), np.min(self.scores)
            
            if verbose:
                # Report average, max, and min
                print(f"\nAverage score: {average}; Max: {max}; Min: {min}")
            return average, max, min
        else: 
            print("No scores have been calculated")
            return None, None, None

In [ ]:
# For reference, the first 41 cubes (from https://oeis.org/A000578) are:
cubes = [0, 1, 8, 27, 64, 125, 216, 343, 512, 729, 1000, 1331, 1728, 2197, 2744, 3375, 4096, 4913, 5832, 6859, 8000, 9261, 10648, 12167, 13824, 15625, 17576, 19683, 21952, 24389, 27000, 29791, 32768, 35937, 39304, 42875, 46656, 50653, 54872, 59319, 64000]
len(cubes)

#### Random Agent ####
An agent which selects any of the four possible actions randomly at each time step:

In [ ]:
# Define random agent class

class RandomAgent(LunarLander):
    '''
    Inherits from lunar lander class
    and acts randomly
    '''
    def __init__(self, name = "random_agent"):
        LunarLander.__init__(self, name)

    def select_action(self, state):
        '''
        Randomly select action
        '''
        # Randomly select action from action space
        action = self.env.action_space.sample()

        return action

#### Simple Reflex Agent ####

This is an agent which can fire its side thrusters in reaction to its current horizontal and angular velocity in relation to threshold constants chosen by the programmer.

In [ ]:
class ReflexAgent(LunarLander):
    '''
    Inherits from lunar lander class, but 
    selects actions according to horizontal 
    and angular position and angular velocity
    '''
    def __init__(self, name = "reflex_agent"):
        LunarLander.__init__(self, name)

    def select_action(self, state):
        '''
        Select action according to horizontal and
        angular position and angular velocity
        '''
        # Set tolerances
        horiz_vel_tolerance = 0.4
        ang_vel_tolerance = 0.001

        # Horizontal velocity
        vx = state[3]

        # Angular velocity
        ang_vel = state[5]

        # If leftward horizontal velocity
        if ((vx < -horiz_vel_tolerance) and
            # and clockwise velocity
            (ang_vel > ang_vel_tolerance)):

            # Then, fire thruster to move right
            action = RIGHT
        
        # If rightward horizontal velocity
        elif ((vx > horiz_vel_tolerance) and
              # If counter-clockwise velocity
              (ang_vel < -ang_vel_tolerance)):
            
            # Then, fire thruster to move left
            action = LEFT

        else:
            # Do nothing
            action = NOOP
        
        return action 

#### Deep Q-Network Agent ####

A deep Q-network agent has the capacity to remember and learn from its previous experiences using a neural network.

##### Replay Memory and Deep Q-Network #####
First, we must define a buffer to remember previous experiences and a neural network for the agent to use for learning.

In [ ]:
# Define Replay Memory Buffer and Deep Q-Network
# Adapted from: https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
# Additional references:
# https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html#building-models-with-pytorch
# https://pytorch.org/docs/stable/notes/randomness.html

import random
from collections import namedtuple, deque
import torch

# Set random seed for reproducibility
torch.manual_seed(0)

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:",device)

# Named tuple to encode a single state transition
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

# Buffer class to hold and randomly sample recent transitions
class ReplayMemory(object):

    def __init__(self, capacity):
        '''
        Initialize buffer with empty double-ended queue
        of size "capacity"
        '''
        self.memory = deque([], maxlen = capacity)

    def push(self, *args):
        '''
        Add a transition to buffer
        '''
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        '''
        Randomly sample batch of transitions from buffer
        '''
        return random.sample(self.memory, batch_size)

    def __len__(self):
        '''
        Returns number of transitions in buffer
        '''
        return len(self.memory)

import torch.nn as nn
import torch.nn.functional as F

# Deep Q-Network
class DQN(nn.Module):

    def __init__(self, features_size, actions_size):
        '''
        Initialize neural network
        '''
        super(DQN, self).__init__()

        # First fully-connected layer: 
        # Input dimensions: 1 x no. of state feature
        # Output dimensions: 1 x 128
        self.fc1 = nn.Linear(features_size, 128)

        # Second fully-connected layer: 
        # Input: 1 x 128; Output: 1 x 128
        self.fc2 = nn.Linear(128, 128)

        # Final fully-connected layer: 
        # Input: 1 x 128; 
        # Output: 1 x no. of possible actions
        self.fc3 = nn.Linear(128, actions_size)

    def forward(self, state):
        '''
        Forward propagation function
        Input: State features (e.g., position, velocity, ...)
        Output: tensor of action values
        '''
        # First layer followed by ReLU activation function
        z = F.relu(self.fc1(state))
        
        # Second layer followed by ReLU activation function
        z = F.relu(self.fc2(z))
        
        # Final output layer
        return self.fc3(z)

##### DQN Agent ####
Now we can define the agent.

In [ ]:
import math
from itertools import count
import torch.optim as optim

In [ ]:
class DQNAgent(LunarLander):
    '''
    Inherits from lunar lander class, but 
    iteratively learns from experience to approximate 
    Q-values from state features with a neural network
    '''
    def __init__(self, name = "dqn_agent", alpha = 0.5, epsilon = 0.1):
        LunarLander.__init__(self, name)
        self.alpha = alpha
        self.epsilon = epsilon
        
    def update(self, state, action, reward, new_state):
        '''
        Updates weights
        '''
        return None

    def select_action(self, state):
        '''
        Select action
        '''
                
        return None

## Results ## 

In this section, I compare the five different models: an agentless lunar lander, a random agent, a simple reflex agent, and a DQN agent.

### Agentless Lunar Lander ###
First, we consider the agentless lunar lander which never fires any of its thrusters and is therefore subject entirely to its environment:

In [ ]:
# Instantiate agentless lunar lander
non_agent = LunarLander()

# Number of episodes to iterate over
episodes = 100

# Iterate over 100 episodes
non_scores = non_agent.iterate(episodes, verbose = False, video = True)

# Print average, max, min
average, max, min = non_agent.score_stats(verbose = True)

[Link to the video of the agentless lander in the 65<sup>th</sup> episode](videos/agentless-episode-64.mp4)

In [ ]:
# Plot agentless lunar lander scores over all episodes

import matplotlib.pyplot as plt

plt.plot(non_scores, label = "Agentless Lander")
plt.axhline(y = 200, color = "r", linestyle = "dashed", label = "Solution Threshold")  
plt.xlim([0,episodes])
plt.ylim([-600,300])
plt.title("Agentless Lunar Lander")
plt.xlabel("Episode")
plt.ylabel("Score")
plt.legend();

In the above graph, we can see that an agentless lunar lander is not sufficient to solve this problem. In the video linked above, we can see the lunar lander simply falling in a slightly curved trajectory toward the ground.

### Random Agent ###

Next, we consider the random agent which selects an action randomly.

In [ ]:
# Instantiate lunar lander random agent
rand_agent = RandomAgent()

# Iterate over episodes
rand_scores = rand_agent.iterate(episodes, verbose = False, video = True)

# Print average, max, min
average, max, min = rand_agent.score_stats(verbose = True)

[Link to the video of the random agent in the 65<sup>th</sup> episode](videos/random_agent-episode-64.mp4)

In [ ]:
# Plot agentless lander and random agent scores over all episodes

plt.plot(non_scores, label = "Agentless Lander")
plt.plot(rand_scores, label = "Random Agent")
plt.axhline(y = 200, color = "r", linestyle = "dashed", label = "Solution Threshold")  
plt.xlim([0,episodes])
plt.ylim([-600,300])
plt.title("Random Agent vs. Agentless Lunar Lander")
plt.xlabel("Episode")
plt.ylabel("Score")
plt.legend();

As the above graph shows, doing nothing is better than acting randomly in this environment. The agentless lunar lander is relatively more consistent and successful compared to the random agent and also has a slightly higher average score. Obviously, neither learn anything nor come remotely close to consistently reaching the solution threshold, as expected.

### Simple Reflex Agent ###

Next, we examine the agent which can reflexively fire its side engines in reaction to its current horizontal and angular velocity mediated by pre-selected threshold constants. Choosing ideal threshold constants requires tedious trial and error on the part of the programmer, rather than the agent. Thus, only the programmer is learning in this scenario.

In [ ]:
# Instantiate lunar lander reflex agent
rflx_agent = ReflexAgent()

# Iterate over episodes
rflx_scores = rflx_agent.iterate(episodes, verbose = False, video = True)

# Print average max, min
average, max, min = rflx_agent.score_stats(verbose = True)

[Link to the video of the reflex agent in the 65<sup>th</sup> episode](videos/reflex_agent-episode-64.mp4)

In [ ]:
# Plot agentless, random, and reflex lander scores over all episodes

plt.plot(non_scores, label = "Agentless Lander")
plt.plot(rand_scores, label = "Random Agent")
plt.plot(rflx_scores, label = "Reflex Agent")
plt.axhline(y = 200, color = "r", linestyle = "dashed", label = "Solution Threshold")  
plt.xlim([0,episodes])
plt.ylim([-600,300])
plt.title("Random Agent, Reflex Agent, and Agentless Lander")
plt.xlabel("Episode")
plt.ylabel("Score")
plt.legend();

In the above graph, we can see that the reflex agent is only slightly more consistent and successful than the agentless lunar lander but still not even close to solving the problem. Additionally, it is clearly not learning, as expected. Landing a lunar rover appears to be more difficult than firing the left and right thrusters based on horizontal and angular velocity. It might be possible to achieve a higher score by comparing various algorithms and parameters through trial and error, but this would be a tedious process, so we would like to find a more expedient way to find the best policy to follow.

### Deep Q-Network Agent ###

Next, we examine the performance of a remembering, learning deep Q-network agent.

In [ ]:
# Instantiate lunar lander DQN agent
dqn_agent = DQNAgent()

# Iterate over episodes
dqn_scores = dqn_agent.iterate(episodes, verbose = False, video = True)

# Print average, max, min
average, max, min = dqn_agent.score_stats(verbose = True)

[Link to the video of the DQN agent in the 65<sup>th</sup> episode](videos/dqn_agent-episode-64.mp4)

In [ ]:
# Plot agentless, random, reflex, and DQN lander scores over all episodes

plt.plot(non_scores, label = "Agentless Lander")
plt.plot(rand_scores, label = "Random Agent")
plt.plot(rflx_scores, label = "Reflex Agent")
plt.plot(dqn_scores, label = "DQN Agent")
plt.axhline(y = 200, color = "r", linestyle = "dashed", label = "Solution Threshold")  
plt.xlim([0,episodes])
plt.ylim([-600,300])
plt.title("All-Model Comparison")
plt.xlabel("Episode")
plt.ylabel("Score")
plt.legend();

Summary

## Discussion ## 

Does it include discussion (what went well or not and why), and suggestions for improvements or future work?

## Suggestions for Future Research ##

Stochasticity: frame skipping, sticky actions that repeat with a given probability (Macado et al., 2018)

Transfer learning

Convolutional network

Arcade Learning Environment (Bellemare, 2023; Bellemare et al., 2013; Machado et al., 2018) simulating classic Atari 2600 games (AtariAge, 2023)

_AtariAge._ (2023). [Website]. [https://atariage.com](https://atariage.com)

Bellemare, M. G. (2023). Arcade Learning Environment. [https://github.com/mgbellemare/Arcade-Learning-Environment](https://github.com/mgbellemare/Arcade-Learning-Environment)

Bellemare, M. G., Naddaf, Y., Veness, J., & Bowling, M. (2013). The Arcade Learning Environment: An evaluation platform for general agents. _Journal of Artificial Intelligence Research, 47,_ 253-279.
[https://doi.org/10.1613/jair.3912](https://doi.org/10.1613/jair.3912)
 
Finally, it is only right to ask: What is the value of crashing a virtual lunar lander model thousands of times in order to eventually land successfully? The answer is it is better to train a far less-expensive virtual model and then transfer that learning to a real-world environment. Thus, the long-term aim is to develop the ability to transfer this virtual knowledge to a real-world environment.


## References ##

Brownlee, J. (2017, July 3). Gentle introduction to the Adam optimization algorithm for deep learning. Machine Learning Mastery. [https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)

_Gymnasium._ (2022). Farama Foundation. [https://gymnasium.farama.org/](https://gymnasium.farama.org/)

Goodfellow, I., Bengio, Y., & Courville, A. (2016). _Deep learning._ MIT Press. [http://www.deeplearningbook.org](http://www.deeplearningbook.org)

Heintz, B. (2021, April 17). _Building models with Pytorch._ Pytorch. [https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html#building-models-with-pytorch](https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html#building-models-with-pytorch)

Huber, P. J. (1964). Robust estimation of a location parameter. _Annals of Statistics, 53_(1), 73–101. [https//doi.org/10.1214/aoms/1177703732](https//doi.org/10.1214/aoms/1177703732)

Karpathy, A. (2016). _Convolutional neural networks for visual recognition._ Stanford University. [https://cs231n.github.io/convolutional-networks/](https://cs231n.github.io/convolutional-networks/)

Machado, M. C., Bellemare, M. G., Talvitie, E., Veness, J., Hausknecht, M., & Bowling, M. (2018). Revisiting the Arcade Learning Environment: Evaluation protocols and open problems for general agents. _Journal of Artificial Intelligence Research, 61,_ 523–562. [https://doi.org/10.1613/jair.5699](https://doi.org/10.1613/jair.5699)

Mnih, V., Kavukcuoglu, K., Silver, D., Rusu, A. A., Veness, J., Bellemare, M. G., Graves, A., Riedmiller, M., Fidjeland, A. K., Ostrovski, G., Petersen, S., Beattie, C., Sadik, A., Antonoglou, I., King, H., Kumaran, D., Wierstra, D., Legg, S., & Hassabis, D. (2015). Human-level control through deep reinforcement learning. _Nature, 518_, 529–533. [https://doi.org/10.1038/nature14236](https://doi.org/10.1038/nature14236)

Matplotlib. (2023). _API reference._ [https://matplotlib.org/stable/api/index](https://matplotlib.org/stable/api/index)

Mnih, V., Puigdomènech Badia, A., Mirza, M., Graves, A., Harley, T., Lillicrap, T. P., Silver, D., & Kavukcuoglu, K. (2016). Asynchronous methods for deep reinforcement learning. _Proceedings of the 33rd International Conference on Machine Learning, 48,_ 1928–1937. [https://proceedings.mlr.press/v48/mniha16.html](https://proceedings.mlr.press/v48/mniha16.html)

Paske, A., & Towers, M. (2023). _Reinforcement learning (DQN) tutorial._ Pytorch. [https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html)

Pytorch. (2023). _Pytorch documentation._ [https://pytorch.org/docs/stable/index.html](https://pytorch.org/docs/stable/index.html)

Russell, S., & Norvig, R. (2022). Artificial intelligence: A modern approach, (4th ed.). Pearson. 

Sutton, R. S., & Barto, A. G. (2018). Reinforcement learning: An introduction. MIT Press.